In [17]:
import pandas as pd
from pandas import DataFrame

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [18]:
path: str = r"D:\Documents\GitHub\UNI_Stellar_Classification\Data\star_classification_only_features.csv"
data: DataFrame = pd.read_csv(path, index_col="index")
data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [19]:
data.head()

,alpha,delta,u,g,r,i,z,redshift,class,plate
index,,,,,,,,,,
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY,5812
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,GALAXY,10445
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,GALAXY,4576
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,GALAXY,9149
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,GALAXY,6121


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99999 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   alpha     99999 non-null  float64
 1   delta     99999 non-null  float64
 2   u         99999 non-null  float64
 3   g         99999 non-null  float64
 4   r         99999 non-null  float64
 5   i         99999 non-null  float64
 6   z         99999 non-null  float64
 7   redshift  99999 non-null  float64
 8   class     99999 non-null  object 
 9   plate     99999 non-null  int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 8.4+ MB


# Transform the target variable into numeric values

In [21]:
target_encoding: dict = {
    "GALAXY": 0,
    "QSO": 1,
    "STAR": 2
}

data["target"] = data["class"].map(lambda x: target_encoding[x])
data.target.unique()

array([0, 1, 2], dtype=int64)

# Rescale all features
for Trees this will not be necessary, however it can improve the performance of Logistic Regression Models.
Thus, a rescaled and non rescaled version of the dataset will be saved.

There are several methods how to rescale data such as Standardization or MinMaxScaling
Both ways will be tried and the logistic regression model will be evaluated on all of them.

In [27]:
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# min max scale
min_max_scaled_data = min_max_scaler.fit_transform(data.drop(["class", "target"], axis=1), data["target"])
mm_scaled_data: DataFrame = DataFrame(data=min_max_scaled_data, columns=min_max_scaler.get_feature_names_out())
mm_scaled_data["target"] = data.target

# standardise
standard_scaled_data = standard_scaler.fit_transform(data.drop(["class", "target"], axis=1), data["target"])
std_scale_data = DataFrame = DataFrame(data=standard_scaled_data, columns=standard_scaler.get_feature_names_out())
std_scale_data["target"] = data.target

In [28]:
mm_scaled_data.head()

,alpha,delta,u,g,r,i,z,redshift,plate,target
0,0.376905,0.503802,0.591347,0.558050,0.535344,0.427665,0.464377,0.091831,0.451592,0.0
1,0.402286,0.491812,0.632603,0.584423,0.646203,0.515986,0.607035,0.112389,0.828841,0.0
2,0.394960,0.534139,0.654888,0.576463,0.546218,0.435729,0.472194,0.093170,0.350949,0.0
3,0.940947,0.180600,0.511384,0.629186,0.596946,0.486717,0.487460,0.134210,0.723312,0.0
4,0.959118,0.392679,0.387463,0.335579,0.337999,0.287021,0.300043,0.017959,0.476753,0.0


In [29]:
std_scale_data.head()

,alpha,delta,u,g,r,i,z,redshift,plate,target
0,-0.434597,0.425517,0.798798,0.806782,0.403953,0.046001,0.013999,0.079549,0.228626,0.0
1,-0.339915,0.363391,1.198064,1.079967,1.584395,1.185087,1.611170,0.277088,1.797912,0.0
2,-0.367244,0.582702,1.413732,0.997513,0.519736,0.150012,0.101520,0.092415,-0.190031,0.0
3,1.669522,-1.249122,0.024940,1.543642,1.059894,0.807601,0.272435,0.486761,1.358932,0.0
4,1.737308,-0.150255,-1.174337,-1.497665,-1.697426,-1.767888,-1.825836,-0.630273,0.333290,0.0


In [30]:
data.head()

,alpha,delta,u,g,r,i,z,redshift,class,plate,target
index,,,,,,,,,,,
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794,GALAXY,5812,0
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136,GALAXY,10445,0
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195,GALAXY,4576,0
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346,GALAXY,9149,0
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123,GALAXY,6121,0


In [25]:
save_path: str = r"D:\Documents\GitHub\UNI_Stellar_Classification\Data\star_classification_preprocessed.csv"
data.drop(["class"], axis=1, inplace=True)
data.to_csv(save_path)

In [31]:
save_path_mm_scale: str = r"D:\Documents\GitHub\UNI_Stellar_Classification\Data\star_classification_preprocessed_min_max_scale.csv"
mm_scaled_data.to_csv(save_path_mm_scale)

In [32]:
save_path_std_scale: str = r"D:\Documents\GitHub\UNI_Stellar_Classification\Data\star_classification_preprocessed_std_scale.csv"
std_scale_data.to_csv(save_path_std_scale)